# Experiment 01: Airline dataset

In this notebook we are going to perform a prediction experiment with XGBoost and LightGBM using the Airline dataset. We are also going to analyze if the dataset has [concept drift](https://en.wikipedia.org/wiki/Concept_drift). 

In this experiment we use [the airline dataset](http://kt.ijs.si/elena_ikonomovska/data.html) to predict arrival delay. The dataset consists of a large amount of records, containing flight arrival and departure details for all the commercial flights within the USA, from October 1987 to April 2008. Its size is around 116 million records and 5.76 GB of memory.

The machine we used for this experiment is a Azure VM Standard DS15 v2 with 20 cores and 140 GB memory. Its operating system is Ubuntu 16.04.

For both XGBoost and LightGBM we compiled from source, to get the last improvements. In XGboost we used the commit `8e2a1ff2bfd29f0d08c117d617fc0837eb6796cc` of May 30th. For LightGBM we used the commit `73968a96829e212b333c88cd44725c8c39c03ad1` of June 2nd. To get these versions and replicate our experiments:
```python
git clone --recursive *url_of_library*
git checkout *oldcommit*
git submodule update --recursive
```

In [20]:
import json
import sys
import warnings

import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import pkg_resources
from IPython.display import SVG, display
from bokeh.io import export_svgs
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from libs.conversion import convert_cols_categorical_to_numeric, convert_related_cols_categorical_to_numeric
from libs.loaders import load_airline
from libs.notebook_memory_management import start_watching_memory
from libs.timer import Timer
from libs.utils import get_number_processors
from lightgbm.sklearn import LGBMRegressor, LGBMClassifier
from sklearn.metrics import (accuracy_score, roc_auc_score, f1_score, precision_score,
                             recall_score)
from libs.metrics import classification_metrics_binary, classification_metrics_binary_prob, binarize_prediction
from toolz import curry
from xgboost import XGBRegressor

print("System version: {}".format(sys.version))
print("XGBoost version: {}".format(pkg_resources.get_distribution('xgboost').version))
print("LightGBM version: {}".format(pkg_resources.get_distribution('lightgbm').version))

warnings.filterwarnings("ignore", category=DeprecationWarning)

System version: 3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
XGBoost version: 0.6
LightGBM version: 0.2


In [2]:
%%time
X_train_path = '/fileshare/airline/X_train_airline.npy'
y_train_path = '/fileshare/airline/y_train_airline.npy'
X_test_path = '/fileshare/airline/X_test_airline.npy'
y_test_path = '/fileshare/airline/y_test_airline.npy'
X_train = np.load(X_train_path)
y_train = np.load(y_train_path)
X_test = np.load(X_test_path)
y_test = np.load(y_test_path)

CPU times: user 88 ms, sys: 5.06 s, total: 5.14 s
Wall time: 4min 23s


The dataset we are going to use in this notebook is huge, therefore we want to monitor the memory consumption. 

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
start_watching_memory()

In [3] used 11.9180 MiB RAM in 0.47s, total RAM usage 224.50 MiB


# 1) XGBoost vs LightGBM benchmark
In the next section we compare both libraries speed, accuracy and other metrics for the dataset of airline arrival delay. 

### Data loading and management

In [4]:
%%time
df_plane = load_airline()
print(df_plane.shape)

INFO:libs.loaders:MOUNT_POINT not found in environment. Defaulting to /fileshare


(115069017, 14)
CPU times: user 2min 2s, sys: 14.9 s, total: 2min 17s
Wall time: 4min 35s
In [4] used 21994.9062 MiB RAM in 275.34s, total RAM usage 22219.41 MiB


In [5]:
df_plane.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,AA,190,247,SFO,ORD,1846,0,27
1,1987,10,1,4,5,114,EA,57,74,LAX,SFO,337,0,5
2,1987,10,1,4,5,35,HP,351,167,ICT,LAS,987,0,17
3,1987,10,1,4,5,40,DL,251,35,MCO,PBI,142,0,-2
4,1987,10,1,4,8,517,UA,500,208,LAS,ORD,1515,0,17


In [5] used 0.3125 MiB RAM in 0.14s, total RAM usage 22219.72 MiB


The first step is to convert the categorical features to numeric features.

In [6]:
%%time
df_plane_numeric = convert_related_cols_categorical_to_numeric(df_plane, col_list=['Origin','Dest'])
del df_plane

CPU times: user 1min 53s, sys: 9.75 s, total: 2min 2s
Wall time: 2min
In [6] used 5270.4609 MiB RAM in 120.17s, total RAM usage 27490.18 MiB


In [7]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,AA,190,247,0,33,1846,0,27
1,1987,10,1,4,5,114,EA,57,74,1,0,337,0,5
2,1987,10,1,4,5,35,HP,351,167,2,4,987,0,17
3,1987,10,1,4,5,40,DL,251,35,3,41,142,0,-2
4,1987,10,1,4,8,517,UA,500,208,4,33,1515,0,17


In [7] used 0.0039 MiB RAM in 0.13s, total RAM usage 27490.18 MiB


In [8]:
%%time
df_plane_numeric = convert_cols_categorical_to_numeric(df_plane_numeric, col_list='UniqueCarrier')

CPU times: user 1min 5s, sys: 8.57 s, total: 1min 14s
Wall time: 1min 12s
In [8] used 13168.7500 MiB RAM in 72.66s, total RAM usage 40658.93 MiB


In [9]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,0,190,247,0,33,1846,0,27
1,1987,10,1,4,5,114,1,57,74,1,0,337,0,5
2,1987,10,1,4,5,35,2,351,167,2,4,987,0,17
3,1987,10,1,4,5,40,3,251,35,3,41,142,0,-2
4,1987,10,1,4,8,517,4,500,208,4,33,1515,0,17


In [9] used 0.0039 MiB RAM in 0.12s, total RAM usage 40658.94 MiB


To simplify the pipeline, we are going to set a classification problem where the goal is to classify wheather a flight has arrived delayed or not. For that we need to binarize the variable `ArrDelay`.

If you want to extend this experiment, you can set a regression problem and try to identify the number of minutes of delay a fight has. Both XGBoost and LightGBM have regression classes.

In [10]:
df_plane_numeric = df_plane_numeric.apply(lambda x: x.astype('int16'))

In [10] used 3072.6914 MiB RAM in 29.99s, total RAM usage 43731.63 MiB


In [11]:
%%time
df_plane_numeric['ArrDelayBinary'] = 1*(df_plane_numeric['ArrDelay'] > 0)

CPU times: user 548 ms, sys: 488 ms, total: 1.04 s
Wall time: 923 ms
In [11] used 877.9219 MiB RAM in 1.03s, total RAM usage 44609.55 MiB


In [12]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay,ArrDelayBinary
0,1987,10,1,4,1,556,0,190,247,0,33,1846,0,27,1
1,1987,10,1,4,5,114,1,57,74,1,0,337,0,5,1
2,1987,10,1,4,5,35,2,351,167,2,4,987,0,17,1
3,1987,10,1,4,5,40,3,251,35,3,41,142,0,-2,0
4,1987,10,1,4,8,517,4,500,208,4,33,1515,0,17,1


In [12] used 0.0039 MiB RAM in 0.12s, total RAM usage 44609.55 MiB


Once the features are prepared, let's split the dataset into train and test set. We won't use validation for this example (however, you can try to add it).

In [13]:
def split_train_val_test_df(df, val_size=0.2, test_size=0.2):
    train, validate, test = np.split(df.sample(frac=1), 
                                     [int((1-val_size-test_size)*len(df)), int((1-test_size)*len(df))])
    return train, validate, test

In [13] used 0.0000 MiB RAM in 0.11s, total RAM usage 44609.55 MiB


In [14]:
%%time
train, validate, test = split_train_val_test_df(df_plane_numeric, val_size=0, test_size=0.2)
print(train.shape)
print(validate.shape)
print(test.shape)

(92055213, 15)
(0, 15)
(23013804, 15)
CPU times: user 38.2 s, sys: 3.68 s, total: 41.9 s
Wall time: 40.9 s
In [14] used 4653.1758 MiB RAM in 41.00s, total RAM usage 49262.73 MiB


In [15]:
def generate_feables(df):
    X = df[df.columns.difference(['ArrDelay', 'ArrDelayBinary'])]
    y = df['ArrDelayBinary']
    return X,y

In [15] used 0.0078 MiB RAM in 0.10s, total RAM usage 49262.74 MiB


In [16]:
%%time
X_train, y_train = generate_feables(train)
X_val, y_val = generate_feables(validate)
X_test, y_test = generate_feables(test)

CPU times: user 1.06 s, sys: 748 ms, total: 1.81 s
Wall time: 1.81 s
In [16] used 2853.2031 MiB RAM in 1.91s, total RAM usage 52115.94 MiB


In [17]:
del train, validate, test

In [17] used 0.0000 MiB RAM in 0.10s, total RAM usage 52115.94 MiB


### Training 
Now we are going to create two pipelines, one of XGBoost and one for LightGBM. The technology behind both libraries is different, so it is difficult to compare them in the exact same model setting. XGBoost grows the trees depth-wise and controls model complexity with `max_depth`. Instead, LightGBM uses a leaf-wise algorithm and controls the model complexity by `num_leaves`. As a tradeoff, we use XGBoost with `max_depth=8`, which will have max number leaves of 255, and compare it with LightGBM with `num_leaves=255`. 

In [3]:
results_dict = dict()

In [4]:
number_processors = get_number_processors()
print(number_processors)

24


Let's start with the XGBoost classifier.

In [5]:
xgb_clf_pipeline = XGBRegressor(max_depth=8,
                                n_estimators=50,
                                min_child_weight=30,
                                learning_rate=0.1,
                                colsample_bytree=0.80,
                                scale_pos_weight=2,
                                gamma=0.1,
                                reg_lambda=1,
                                subsample=1,
                                n_jobs=number_processors,
                                random_state=77)

In [6]:
with Timer() as t:
    xgb_clf_pipeline.fit(X_train, y_train)

In [7]:
results_dict['xgb']={
    'train_time': t.interval
}

Training XGBoost model with leave-wise growth

In [8]:
xgb_hist_clf_pipeline = XGBRegressor(max_depth=0,
                                    n_estimators=50,
                                    min_child_weight=30,
                                    learning_rate=0.1,
                                    colsample_bytree=0.80,
                                    scale_pos_weight=2,
                                    gamma=0.1,
                                    reg_lambda=1,
                                    subsample=1,
                                    max_leaves=255,
                                    grow_policy='lossguide',
                                    tree_method='hist',
                                    n_jobs=number_processors,
                                    random_state=77)

In [9]:
with Timer() as t:
    xgb_hist_clf_pipeline.fit(X_train, y_train)

In [10]:
results_dict['xgb_hist']={
    'train_time': t.interval
}

Training LightGBM model

In [11]:
lgbm_clf_pipeline = LGBMRegressor(num_leaves=255,
                                  n_estimators=50,
                                  min_child_weight=30,
                                  learning_rate=0.1,
                                  colsample_bytree=0.80,
                                  scale_pos_weight=2,
                                  min_split_gain=0.1,
                                  reg_lambda=1,
                                  subsample=1,
                                  nthread=number_processors,
                                  seed=77)

In [12]:
with Timer() as t:
    lgbm_clf_pipeline.fit(X_train, y_train)

In [13]:
results_dict['lgbm']={
    'train_time': t.interval
}

As it can be seen in the results, given the specific versions and parameters of both XGBoost and LightGBM and in this specific dataset, LightGBM is faster. 

In general terms, leaf-wise algorithms are more efficient, they converge much faster than depth-wise. However, it may cause over-fitting when the data is small or there are too many leaves.

### Evaluation
Now let's evaluate the model in the test set.

In [14]:
with Timer() as t:
    y_prob_xgb = np.clip(xgb_clf_pipeline.predict(X_test), 0.0001, 0.9999)

In [15]:
results_dict['xgb']['test_time'] = t.interval

In [16]:
with Timer() as t:
    y_prob_xgb_hist = np.clip(xgb_hist_clf_pipeline.predict(X_test), 0.0001, 0.9999)

In [17]:
results_dict['xgb_hist']['test_time'] = t.interval

In [18]:
with Timer() as t:
    y_prob_lgbm = np.clip(lgbm_clf_pipeline.predict(X_test), 0.0001, 0.9999)

In [19]:
results_dict['lgbm']['test_time'] = t.interval

### Metrics
We are going to obtain some metrics to evaluate the performance of each of the models.

In [22]:
y_pred_xgb = binarize_prediction(y_prob_xgb)
y_pred_xgb_hist = binarize_prediction(y_prob_xgb_hist)
y_pred_lgbm = binarize_prediction(y_prob_lgbm)

In [23]:
report_xgb = classification_metrics_binary(y_test, y_pred_xgb)
report2_xgb = classification_metrics_binary_prob(y_test, y_prob_xgb)
report_xgb.update(report2_xgb)

In [24]:
results_dict['xgb']['performance'] = report_xgb

In [25]:
report_xgb_hist = classification_metrics_binary(y_test, y_pred_xgb_hist)
report2_xgb_hist = classification_metrics_binary_prob(y_test, y_prob_xgb_hist)
report_xgb_hist.update(report2_xgb_hist)

In [26]:
results_dict['xgb_hist']['performance'] = report_xgb_hist

In [27]:
report_lgbm = classification_metrics_binary(y_test, y_pred_lgbm)
report2_lgbm = classification_metrics_binary_prob(y_test, y_prob_lgbm)
report_lgbm.update(report2_lgbm)

In [28]:
results_dict['lgbm']['performance'] = report_lgbm

In [29]:
# Results
print(json.dumps(results_dict, indent=4, sort_keys=True))

{
    "lgbm": {
        "performance": {
            "AUC": 0.8089804506704468,
            "Accuracy": 0.7356989309546567,
            "F1": 0.6944528933896273,
            "Precision": 0.7693635631257912,
            "Recall": 0.6328355547108336
        },
        "test_time": 11.604696890000014,
        "train_time": 249.0863602190002
    },
    "xgb": {
        "performance": {
            "AUC": 0.7851551960872168,
            "Accuracy": 0.6293107388939264,
            "F1": 0.6975562844735109,
            "Precision": 0.5691896152961613,
            "Recall": 0.9006831352646643
        },
        "test_time": 9.702182734000417,
        "train_time": 5067.751858128
    },
    "xgb_hist": {
        "performance": {
            "AUC": 0.8071906630740422,
            "Accuracy": 0.6730354529829141,
            "F1": 0.716584754804147,
            "Precision": 0.608720541639964,
            "Recall": 0.8709081532478107
        },
        "test_time": 10.452880827999252,
        "trai

The experiment shows a fairly similar performance in both libraries, being LightGBM slightly better.

In [46]:
del xgb_clf_pipeline, xgb_hist_clf_pipeline, lgbm_clf_pipeline, X_train, X_test, X_val

In [46] used -49294.8203 MiB RAM in 1.58s, total RAM usage 48220.56 MiB


# 2) Concept drift
In this section we are trying to find concept drift in the dataset to check if retraining is valuable.

### Data management
We are going to pack the data yearly to try to find concept drift

In [47]:
initial_year = 1987
num_ini = 5

In [47] used 0.0000 MiB RAM in 0.10s, total RAM usage 48220.56 MiB


In [48]:
def generate_subset_by_year(df, year_ini, year_end):
    return df[df['Year'].isin(range(year_ini, year_end))]

In [48] used 0.0000 MiB RAM in 0.10s, total RAM usage 48220.56 MiB


In [49]:
%%time
subset_base = generate_subset_by_year(df_plane_numeric, initial_year, initial_year + num_ini)
print(subset_base.shape)

(16810190, 15)
CPU times: user 1.73 s, sys: 448 ms, total: 2.18 s
Wall time: 2.18 s
In [49] used 577.0703 MiB RAM in 2.29s, total RAM usage 48797.63 MiB


In [50]:
%%time
rest_df = df_plane_numeric.loc[df_plane_numeric.index.difference(subset_base.index)]
print(rest_df.shape)

(98258827, 15)
CPU times: user 1min 13s, sys: 7.52 s, total: 1min 20s
Wall time: 1min 21s
In [50] used 9696.6016 MiB RAM in 81.20s, total RAM usage 58494.23 MiB


### Traininig
Let's see what happens when we train on a subset of data and then evaluate in the data of the following years.

In [51]:
X_train, y_train = generate_feables(subset_base)
del(subset_base)

In [51] used -285.5078 MiB RAM in 0.33s, total RAM usage 58208.73 MiB


In [52]:
clf = LGBMClassifier(num_leaves=255,
                    n_estimators=100,
                    min_child_weight=30,
                    learning_rate=0.1,
                    subsample=0.80,
                    colsample_bytree=0.80,
                    seed=42)

In [52] used 0.0000 MiB RAM in 0.10s, total RAM usage 58208.73 MiB


In [53]:
%%time
clf.fit(X_train, y_train)

CPU times: user 23min 54s, sys: 1min 58s, total: 25min 52s
Wall time: 1min 20s


LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.8, learning_rate=0.1,
        max_bin=255, max_depth=-1, min_child_samples=10,
        min_child_weight=30, min_split_gain=0, n_estimators=100,
        nthread=-1, num_leaves=255, objective='binary', reg_alpha=0,
        reg_lambda=0, seed=42, silent=True, subsample=0.8,
        subsample_for_bin=50000, subsample_freq=1)

In [53] used 1029.5625 MiB RAM in 80.90s, total RAM usage 59238.29 MiB


### Prediction

In [54]:
@curry
def predict_accuracy(clf, test_df):
    X_test, y_test = generate_feables(test_df)
    y_pred = clf.predict(X_test)
    return accuracy_score(y_test, y_pred)

In [54] used 0.0000 MiB RAM in 0.10s, total RAM usage 59238.29 MiB


In [55]:
%%time
accuracy_series = rest_df.groupby('Year').apply(predict_accuracy(clf))
print(accuracy_series)

Year
1992    0.755851
1993    0.755262
1994    0.743402
1995    0.730876
1996    0.722213
1997    0.720379
1998    0.705320
1999    0.700228
2000    0.689134
2001    0.673514
2002    0.679002
2003    0.684676
2004    0.679677
2005    0.672613
2006    0.663230
2007    0.651053
2008    0.635933
dtype: float64
CPU times: user 36min 44s, sys: 50.6 s, total: 37min 34s
Wall time: 2min 33s
In [55] used 3526.0625 MiB RAM in 153.68s, total RAM usage 62764.35 MiB


From the results we can observe that the accuracy of the model gets worse as the years pass on.

### Retraining
Now let's see what happens when we retrain and evaluate in the data of the following years.

In [56]:
new_init = 15

In [56] used 0.0000 MiB RAM in 0.10s, total RAM usage 62764.35 MiB


In [57]:
%%time
subset_retrain = generate_subset_by_year(df_plane_numeric, initial_year, initial_year + new_init) 
print(subset_retrain.shape)

(69425349, 15)
CPU times: user 4.5 s, sys: 2.55 s, total: 7.05 s
Wall time: 7.08 s
In [57] used 2913.2109 MiB RAM in 7.19s, total RAM usage 65677.56 MiB


In [58]:
X_train, y_train = generate_feables(subset_retrain)

In [58] used 1304.6211 MiB RAM in 1.03s, total RAM usage 66982.18 MiB


In [59]:
clf_retrain = LGBMClassifier(num_leaves=255,
                            n_estimators=100,
                            min_child_weight=30,
                            learning_rate=0.1,
                            subsample=0.80,
                            colsample_bytree=0.80,
                            seed=42)

In [59] used 0.0000 MiB RAM in 0.10s, total RAM usage 66982.18 MiB


In [60]:
%%time
clf_retrain.fit(X_train, y_train)

CPU times: user 1h 38min 54s, sys: 2min 22s, total: 1h 41min 17s
Wall time: 5min 20s


LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.8, learning_rate=0.1,
        max_bin=255, max_depth=-1, min_child_samples=10,
        min_child_weight=30, min_split_gain=0, n_estimators=100,
        nthread=-1, num_leaves=255, objective='binary', reg_alpha=0,
        reg_lambda=0, seed=42, silent=True, subsample=0.8,
        subsample_for_bin=50000, subsample_freq=1)

In [60] used 4641.9883 MiB RAM in 320.69s, total RAM usage 71624.17 MiB


### Prediction

In [61]:
%%time
rest_df = df_plane_numeric.loc[df_plane_numeric.index.difference(subset_retrain.index)]
print(rest_df.shape)

(45643668, 15)
CPU times: user 29 s, sys: 4.3 s, total: 33.3 s
Wall time: 32.5 s
In [61] used -1183.8164 MiB RAM in 32.58s, total RAM usage 70440.36 MiB


In [62]:
%%time
accuracy_retrain = rest_df.groupby('Year').apply(predict_accuracy(clf_retrain))
print(accuracy_retrain)

Year
2002    0.751166
2003    0.749844
2004    0.726010
2005    0.720412
2006    0.707695
2007    0.696212
2008    0.699735
dtype: float64
CPU times: user 16min 4s, sys: 30.5 s, total: 16min 34s
Wall time: 1min 10s
In [62] used -1114.4883 MiB RAM in 70.26s, total RAM usage 69325.87 MiB


### Plot

In [63]:
def plot_metrics(metric1, metric2, legend1=None, legend2=None, x_label=None, y_label=None):
    lists = sorted(metric1.items()) 
    x, y = zip(*lists) 
    fig, ax = plt.subplots()
    ax.plot(x, y, label=legend1, color='#5975a4')
    lists2 = sorted(metric2.items()) 
    x2, y2 = zip(*lists2) 
    ax.plot(x2, y2, label=legend2, color='#5f9e6f')
    legend = ax.legend(loc=0)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    plt.show()
    return ax

In [63] used 0.0000 MiB RAM in 0.11s, total RAM usage 69325.87 MiB


In [64]:
from bokeh.models.sources import ColumnDataSource
from bokeh.models import HoverTool

In [64] used 0.0000 MiB RAM in 0.10s, total RAM usage 69325.87 MiB


In [65]:
data_cds = ColumnDataSource(pd.DataFrame({
    'train_acc': accuracy_series,
    'retrain_acc': accuracy_retrain
}))

In [65] used 0.2422 MiB RAM in 0.11s, total RAM usage 69326.11 MiB


In [66]:
# Airline Retrain Results
p = figure(y_axis_label='Accuracy', plot_width=700, plot_height=350, tools="pan,wheel_zoom,box_zoom,reset")
l1 = p.line('Year', 'train_acc', legend=' Train AUC', line_color="#5975a4", source=data_cds, line_width=6, line_cap="round")
p.line('Year', 'retrain_acc', legend=' Retrain AUC', line_color="#a1bae3", source=data_cds, line_width=6, line_cap="round")
l1_hover = HoverTool(renderers=[l1], tooltips=[( 'Train',  '@{train_acc}{0.4f}' ), ( 'Retrain',  '@{retrain_acc}{0.4f}' )], mode='vline')
p.add_tools(l1_hover)
show(p)

In [66] used -1002.8867 MiB RAM in 0.24s, total RAM usage 68323.22 MiB


In [67]:
# SVG: Airline Retrain Results
# Save the plot and display it so that is is visible on Github
p.output_backend = "svg"
export_svgs(p, filename="airline_retrain.svg")
display(SVG('airline_retrain.svg'))

In [67] used -19486.5000 MiB RAM in 2.69s, total RAM usage 48836.72 MiB


As it can be seen, the performance is better after retraining. We have found concept drift in this dataset.